We recently launched a new measure of [seven daya prescribing for long term conditions](https://openprescribing.net/measure/seven_day_prescribing/all-england/) where we found variation across the country. In the background we state that there is no consensus on duration of long term prescriptions and guidance varies throughout the country around stable long term prescriptions. Most CCGs will have some form of guidance generally for one, two or three months. 

The University of Bristol have produced a [policy briefing](https://www.bristol.ac.uk/policybristol/policy-briefings/repeat-prescription-costs/) arguing that three month repeat prescriptions are more cost-effective. They base this on a systematic review and studies in CPRD. 

I have had a quick skim of the systematic review (based exclusively on American data) and the CPRD. The policy briefing states that _current guidance to issue 28-day repeat prescriptions_. This deviates from our assertation in our seven days measure. This notebook seeks to ascertain what the variation is in 28 v 56 v 84 across the country for our bask of medicines commonly prescribed once daily for long term conditions.

In [1]:
##importing libraries that are need to support analysis
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [6]:
### here we extract data for modelling
sql = '''
SELECT
  pct,
  items,
  quantity_per_item
FROM
 ebmdatalab.hscic.raw_prescribing_normalised AS presc
JOIN
  hscic.ccgs AS ccgs
ON
presc.pct=ccgs.code
WHERE
(bnf_code LIKE "0205051R0%" OR  ##ramipril
bnf_code LIKE "0212000B0%" OR ##atrovastatin
bnf_code LIKE "0212000Y0%" OR ##simvastatin
bnf_code LIKE "0602010V0%" OR ##levothyroxine
bnf_code LIKE "0206020A0%") ##amlodipine
AND
(bnf_name LIKE '%_Tab%' or bnf_name LIKE '%_Cap%') ##this restricts to tablets or capsules
AND (month >= '2018-08-01'
    AND month <= '2019-07-01')
AND
ccgs.org_type='CCG'
GROUP BY
  pct,
  items,
  quantity_per_item
    '''

df_ltc = bq.cached_read(sql, csv_path='ltc_qty.csv')
df_ltc.head(10)

,pct,items,quantity_per_item
0,01C,1,24
1,01C,2,12
2,01C,1,5
3,01C,1,20
4,01C,1,18
5,01C,2,20
6,01C,2,45
7,01C,1,72
8,01C,1,98
9,01C,4,12


In [3]:
df_ltc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191407 entries, 0 to 191406
Data columns (total 3 columns):
pct                  191407 non-null object
items                191407 non-null int64
quantity_per_item    191407 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.4+ MB


In [4]:
df_rx_repeat = df_ltc.groupby(['pct', 'total_quantity'])['items'].sum().reset_index()
df_rx_repeat.head(30)

KeyError: 'total_quantity'

In [ ]:
plt.figure(figsize=(15,10))
df_rx_repeat.plot.bar(x='total_quantity', y='items')
plt.show()